# Part 2

## Tutorial Part 2: Computer Vision with TransferLearning

**Transfer Learning** - storing knowledge gained while solving one problem and applying it to a different but related problem.
> [More information](https://en.wikipedia.org/wiki/Transfer_learning)

TensorFlow has good selection of pre-trained models that can be imported right in TensorFlow model. It is called [*TensorFlow Hub*](https://tfhub.dev).
As a compliment to main framework Google has published also additional package called [*TensorFlow Datasets*](https://www.tensorflow.org/datasets/catalog/overview#all_datasets) which has collection of the most popular datasets.

In this part of tutorial we will use [*The Standford Dogs dataset*](https://www.tensorflow.org/datasets/catalog/stanford_dogs) imported through TensorFlow datasets.

## Check if gpu is available and TF version

In [82]:
import tensorflow as tf
tf.__version__

'2.9.2'

In [83]:
!nvidia-smi

Mon Oct 24 16:53:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    30W /  70W |  14632MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get data

In [84]:
import tensorflow_datasets as tfds

In [85]:
(train_data, test_data), ds_info = tfds.load(name='stanford_dogs',
                                             split=['train', 'test'],
                                             shuffle_files=True,
                                             as_supervised=True,
                                             with_info=True,
                                             batch_size=32)

In [86]:
ds_info

tfds.core.DatasetInfo(
    name='stanford_dogs',
    full_name='stanford_dogs/0.2.0',
    description="""
    The Stanford Dogs dataset contains images of 120 breeds of dogs from around
    the world. This dataset has been built using images and annotation from
    ImageNet for the task of fine-grained image categorization. There are
    20,580 images, out of which 12,000 are used for training and 8580 for
    testing. Class labels and bounding box annotations are provided
    for all the 12,000 images.
    """,
    homepage='http://vision.stanford.edu/aditya86/ImageNetDogs/main.html',
    data_path='~/tensorflow_datasets/stanford_dogs/0.2.0',
    file_format=tfrecord,
    download_size=778.12 MiB,
    dataset_size=744.72 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'image/filename': Text(shape=(), dtype=tf.string),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=120),
        'objects': Sequence({
           

In [87]:
train_data, test_data

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

## Prepare data

### Batch and prefetch

In [88]:
import tensorflow as tf

In [89]:
# train_data = train_data.batch(32).prefetch(tf.data.AUTOTUNE)
# test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)
# TFDS already batched dataset for us
train_data = train_data.map(lambda image, label: (tf.image.resize(image, (224, 224)), label)).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(lambda image, label: (tf.image.resize(image, (224, 224)), label)).prefetch(tf.data.AUTOTUNE)
len(train_data), len(test_data)

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

**Note:**

* **Batch size** - the number of samples that are passed to the network at once.
![Training with large minibatches is bad for your heath](https://github.com/fulcrum101/Problemcon_hackathon_TensorFlow/blob/master/docs/assets/tweet_2.png?raw=1)
[Revisiting Small Batch Training for deep Neural Networks paper](https://arxiv.org/abs/1804.07612)
* **Prefetching** overlaps the preprocessing and model execution of a training step.
> On the step `s`, the input pipeline is reading the data for step `s+1`.
>
> `tf.data.AUTOTUNE` tunes value dynamically at runtime.

[More information](https://www.tensorflow.org/guide/data_performance)

### Data Augmentation

*Data Augmentation* is important concept against *overfitting* problem.
* **Data Augmentation** - a technique to increase the diversity of your training set by applying random (but realistic) transformations, such as image rotation.
> [More Information](https://www.tensorflow.org/tutorials/images/data_augmentation)
* **Overfiting** -  concept in data science, which occurs when a statistical model fit exactly against its training data
> [More Information](https://www.ibm.com/cloud/learn/overfitting)

In [90]:
# Build data augmentation layer
# Note: in TensorFlow models can be used as layers
from tensorflow.keras import layers

data_augmentation = tf.keras.models.Sequential([
    layers.RandomHeight(0.2), # https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomHeight
    layers.RandomWidth(0.2), # https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomWidth
    layers.RandomFlip(), # https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomFlip
    layers.RandomZoom(0.2), # https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomZoom
    layers.RandomRotation(0.2) # https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomRotation
], name='data_augmentation')

## Build model

### Get pretrained model

Links:
- [Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet/EfficientNetB0)

- [TF Hub](https://tfhub.dev/google/collections/efficientnet/1)

In [91]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False # Freeze model's weigth

### Callbacks
* `ModelCheckpoint` - saves model or model weights at some frequency.
> [More information](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint)
* `EarlyStopping` - stops training when a monitored metric has stopped improving.
> [More information](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping)

In [92]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='model_checkpoints/checkpoint.ckpt',
                                                         save_weights_only=True,
                                                         save_best_only=True,
                                                         save_freq='epoch',
                                                         verbose=1)

In [93]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                           verbose=1,
                                                           restore_best_weights=True,
                                                           patience=5)

### Create a model with Functional API

In [94]:
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name='input_layer')
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name='global_average_pooling_layer')(x)
outputs = tf.keras.layers.Dense(120, activation='softmax', name='output_layer')(x)

In [95]:
model = tf.keras.models.Model(inputs, outputs, name='cv_model')
model.summary()

Model: "cv_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pooling_laye  (None, 1280)             0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 output_layer (Dense)        (None, 120)               153720    
                                                                 
Total params: 4,203,291
Trainable params: 153,720
Non-trai

In [96]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [97]:
model.fit(train_data,
          validation_data=test_data,
          validation_steps=int(0.2*len(test_data)),
          epochs=100,
          callbacks=[early_stopping_callback,
                     checkpoint_callback],
          verbose=1)

Epoch 1/100
375/375 [==============================] - ETA: 0s - loss: 3.8025 - accuracy: 0.1755
Epoch 1: val_loss improved from inf to 2.24280, saving model to model_checkpoints/checkpoint.ckpt
375/375 [==============================] - 89s 221ms/step - loss: 3.8025 - accuracy: 0.1755 - val_loss: 2.2428 - val_accuracy: 0.4900
Epoch 2/100
375/375 [==============================] - ETA: 0s - loss: 3.0469 - accuracy: 0.3007
Epoch 2: val_loss improved from 2.24280 to 1.78028, saving model to model_checkpoints/checkpoint.ckpt
375/375 [==============================] - 78s 206ms/step - loss: 3.0469 - accuracy: 0.3007 - val_loss: 1.7803 - val_accuracy: 0.5590
Epoch 3/100
375/375 [==============================] - ETA: 0s - loss: 2.8219 - accuracy: 0.3380
Epoch 3: val_loss improved from 1.78028 to 1.69082, saving model to model_checkpoints/checkpoint.ckpt
375/375 [==============================] - 76s 201ms/step - loss: 2.8219 - accuracy: 0.3380 - val_loss: 1.6908 - val_accuracy: 0.5696
Epoch

In [100]:
model.evaluate(test_data) # Not the best practice, validation dataset should be used here

269/269 [==============================] - 27s 102ms/step - loss: 1.5423 - accuracy: 0.5885


[1.5422791242599487, 0.5884615182876587]

[Different models with Stanfor Dogs Dataset](https://paperswithcode.com/dataset/stanford-dogs)

## How we can improve our model?
- Get more data
- Try different architecture (more complex or simpler)
- Train for Longer

So if you want reach good results - experiment, experiment, experiment!
Good luck!

## More model examples:

One more example of Computer Vision classifier can be found [here](https://github.com/fulcrum101/MIPT_project_improved/blob/main/model.ipynb) with 99.27% accuracy, [AlexNet](https://paperswithcode.com/method/alexnet) ([Convolutional Neural Network (CNN)](https://paperswithcode.com/methods/category/convolutional-neural-networks)).

Creating image `tf.data.Dataset` from folders with images can be found there as well. [More information.](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)